In [1]:
class Point2D:
    _fields = ['x', 'y']
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
    
class Point3D:
    _fields = ['x', 'y', 'z']
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

In [2]:
class SlottedStruct(type):
    def __new__(cls, name, bases, class_dict):
        cls_object = super().__new__(cls, name, bases, class_dict)
        
        # setup the __slots__
        setattr(cls_object, '__slots__', [f'_{field}' for field in cls_object._fields])
            
        # create read-only property for each field
        for field in cls_object._fields:
            slot = f'_{field}'
            # this will not work!
            # remember about how closures work! The free variable is resolved when the function is called!
            #     setattr(cls_object, field, property(fget=lambda self: getattr(self, slot)))
            # so instead we have to use this workaround, by specifying the slot as a defaulted argument
            setattr(cls_object, field, property(fget=lambda self, attrib=slot: getattr(self, attrib)))

        return cls_object

In [3]:
class Person(metaclass=SlottedStruct):
    _fields = ['name', 'age']
    
    def __init__(self, name, age):
        self._name = name
        self._age = age

In [4]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '_fields': ['name', 'age'],
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              '__slots__': ['_name', '_age'],
              'name': <property at 0x1d163899598>,
              'age': <property at 0x1d163899b88>})

In [5]:
class SlottedStruct(type):
    def __new__(cls, name, bases, class_dict):
        cls_object = super().__new__(cls, name, bases, class_dict)
        
        # setup the __slots__
        setattr(cls_object, '__slots__', [f'_{field}' for field in cls_object._fields])
            
        # create read-only property for each field
        for field in cls_object._fields:
            slot = f'_{field}'
            # this will not work!
            #     setattr(cls_object, field, property(fget=lambda self: getattr(self, slot)))
            # Remember about how closures work! The free variable is resolved when the function is called!
            # So instead we have to use this workaround, by specifying the slot as a defaulted argument
            setattr(cls_object, field, property(fget=lambda self, attrib=slot: getattr(self, attrib)))

        # create __eq__ method
        def eq(self, other):
            if isinstance(other, cls_object):
                # ensure each corresponding field is equal
                self_fields = [getattr(self, field) for field in cls_object._fields]
                other_fields = [getattr(other, field) for field in cls_object._fields]
                return self_fields == other_fields
            return False
        setattr(cls_object, '__eq__', eq)

        # create __hash__ method
        def hash_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            return hash(tuple(field_values))
        setattr(cls_object, '__hash__', hash_)
        
        # create __str__ method
        def str_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            field_values_joined = ', '.join(map(str, field_values))  # make every value a string
            return f'{cls_object.__name__}({field_values_joined})'
        setattr(cls_object, '__str__', str_)
        
        # create __repr__ method
        def repr_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            field_key_values = (f'{key}={value}' for key, value in zip(cls_object._fields, field_values))
            field_key_values_str = ', '.join(field_key_values)
            return f'{cls_object.__name__}({field_key_values_str})'
        setattr(cls_object, '__repr__', repr_)
        
        return cls_object

In [6]:
class Person(metaclass=SlottedStruct):
    _fields = ['name']
    
    def __init__(self, name):
        self._name = name

In [7]:
p1 = Person('Alex')
p2 = Person('Alex')

In [8]:
p1==p2

True

In [9]:
class Point2D(metaclass=SlottedStruct):
    _fields = ['x', 'y']
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
        
class Point3D(metaclass=SlottedStruct):
    _fields = ['x', 'y', 'z']
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

In [10]:
p1 = Point2D(1, 2)
p2 = Point2D(1, 2)
p3 = Point2D(0, 0)

In [11]:
p1 is p2, p1 == p2

(False, True)

In [12]:
p1 is p3, p1 == p3

(False, False)

In [13]:
p1 = Point3D(1, 2, 3)
p2 = Point3D(1, 2, 3)
p3 = Point3D(0, 0, 0)

In [15]:
class Singleton(type):
    def __call__(cls, *args, **kwargs):
        print(f'Request received to create an instance of class: {cls}...')
        if getattr(cls, 'existing_instance', None) is None:
            print('Creating instance for the first time...')
            setattr(cls, 'existing_instance', super().__call__(*args, **kwargs))
        else:
            print('Using existing instance...')
        return getattr(cls, 'existing_instance')

In [16]:
class Hundred(metaclass=Singleton):
    value = 100

In [17]:
h1 = Hundred()

Request received to create an instance of class: <class '__main__.Hundred'>...
Creating instance for the first time...


In [18]:
h2=Hundred()

Request received to create an instance of class: <class '__main__.Hundred'>...
Using existing instance...


## workin out through inheritence

In [19]:
class Singleton(type):
    instances = {}
    
    def __call__(cls, *args, **kwargs):
        print(f'Request received to create an instance of class: {cls}...')
        existing_instance = Singleton.instances.get(cls, None)
        if existing_instance is None:
            print('Creating instance for the first time...')
            existing_instance = super().__call__(*args, **kwargs)
            Singleton.instances[cls] = existing_instance
        else:
            print('Using existing instance...')
        return existing_instance

In [20]:
class Hundred(metaclass=Singleton):
    value = 100
    
class Thousand(metaclass=Singleton):
    value = 1000
    
class HundredFold(Hundred):
    value = 100 * 100

In [21]:
h1 = Hundred()
h2 = Hundred()

Request received to create an instance of class: <class '__main__.Hundred'>...
Creating instance for the first time...
Request received to create an instance of class: <class '__main__.Hundred'>...
Using existing instance...


In [22]:
t1 = Thousand()
t2 = Thousand()

Request received to create an instance of class: <class '__main__.Thousand'>...
Creating instance for the first time...
Request received to create an instance of class: <class '__main__.Thousand'>...
Using existing instance...


In [23]:
hf1 = HundredFold()
hf2 = HundredFold()

Request received to create an instance of class: <class '__main__.HundredFold'>...
Creating instance for the first time...
Request received to create an instance of class: <class '__main__.HundredFold'>...
Using existing instance...
